In [1]:
__author__='Monika Soraisam'
__email__='monika.soraisam@noirlab.edu'

In [2]:
from antares_client.search import get_by_id, get_by_ztf_object_id
import numpy as np
from astropy import units as u

In ANTARES lingo, an astronomical target/object is also referred to as a *locus*.<br> 
When pulling the information of a target/locus from the ANTARES database, we can use either the *ANTARES-generated* ID (an alphanumeric ID that starts with *ANT*) or the ID from the ZTF survey. Note that ANTARES is currently serving data from the ongoing ZTF survey.  

**Below, let's retrieve information of this target https://antares.noirlab.edu/loci/ANT20235eaqaaecpcrm via its ANTARES ID.**

In [3]:
# Lookup by ANTARES ID
locus = get_by_id("ANT20235eaqaaecpcrm")

In [4]:
# Here is a summary of some handy properties of this locus -- returns a nested dictionary, where the keys are generally self-explanatory 

print (f'properties of this locus:\n') 
locus.properties

properties of this locus:



{'num_alerts': 26,
 'num_mag_values': 8,
 'brightest_alert_id': 'ztf_candidate:2437157246315015020',
 'brightest_alert_magnitude': 18.604951858520508,
 'brightest_alert_observation_time': 60191.157245399896,
 'newest_alert_id': 'ztf_candidate:2437157246315015020',
 'newest_alert_magnitude': 18.604951858520508,
 'newest_alert_observation_time': 60191.157245399896,
 'oldest_alert_id': 'ztf_candidate:2424258846315015011',
 'oldest_alert_magnitude': 18.821800231933594,
 'oldest_alert_observation_time': 60178.258842600044,
 'is_corrected': 'false',
 'ztf_object_id': 'ZTF23aazjwtl',
 'ztf_ssnamenr': 'null',
 'survey': {'ztf': {'id': ['ZTF23aazjwtl'],
   'rcid': [63],
   'field': [826],
   'ssnamenr': ['null']}},
 'feature_amplitude_magn_r': 0.38857364654541016,
 'feature_anderson_darling_normal_magn_r': 0.12289039611318825,
 'feature_beyond_1_std_magn_r': 0.4,
 'feature_beyond_2_std_magn_r': 0.0,
 'feature_cusum_magn_r': 0.38748149577005836,
 'feature_eta_e_magn_r': 1.266384321927523,
 'feat

In [5]:
# To access the co-ordinates of this locus (format is `astropy.coordinates.sky_coordinate.SkyCoord`). Co-ordinates are not folded in with 
locus.coordinates

<SkyCoord (ICRS): (ra, dec) in deg
    (268.58058154, 64.22561674)>

In [6]:
# If we want to display the RA and DEC as HMS and DMS
print (f'RA {locus.coordinates.ra.to_string(unit=u.hour, sep=":", precision=2)}, DEC {locus.coordinates.dec.to_string(unit=u.deg, sep=":", precision=2)}')

RA 17:54:19.34, DEC 64:13:32.22


In [7]:
# ANTARES, when processing the target (i.e., running through various science filtering algorithms), cross-matches it with a number of legacy catalogs. To access the result of the cross-matching
print (f'The locus has matches/counterparts in these legacy catalogs: {locus.catalogs}')

The locus has matches/counterparts in these legacy catalogs: ['2mass_xsc', 'tns_public_objects']


In [8]:
# We can also access the contents/properties of the matched object in a given legacy catalog. They are in the form of a dictionary
print ('The entries of those matched objects in their respective legacy catalogs: ')
locus.catalog_objects

The entries of those matched objects in their respective legacy catalogs: 


defaultdict(list,
            {'tns_public_objects': [{'objid': 134431,
               'creationdate': '2023-08-18 11:13:04',
               'declination': 64.2256052,
               'discmagfilter': 71.0,
               'discoverydate': '2023-08-18 07:53:51.648000',
               'discoverymag': 18.538000106811523,
               'internal_names': 'ATLAS23rjk, ZTF23aazjwtl',
               'lastmodified': '2023-08-26 15:25:37',
               'name': '2023psq',
               'name_prefix': 'AT',
               'ra': 268.580536,
               'redshift': None,
               'reporting_group': 'ATLAS',
               'reporting_groupid': '18',
               'reports': None,
               'source_group': 'ATLAS',
               'source_groupid': '18',
               'time_received': '2023-08-18 11:13:03',
               'type': '',
               'typeid': ''}],
             '2mass_xsc': [{'sid': 830149,
               'cx': -0.010768779374025,
               'cy': -0.4347077632904

In [9]:
print (f'The TNS_ID of ANTARES target {locus.locus_id} with ZTF_ID {locus.properties["ztf_object_id"]} is {locus.catalog_objects["tns_public_objects"][0]["name"]}')

The TNS_ID of ANTARES target ANT20235eaqaaecpcrm with ZTF_ID ZTF23aazjwtl is 2023psq


## We can also create the same `locus` object and retrieve the information of the target using its ZTF id

In [10]:
# Lookup by ZTF ID
locus = get_by_ztf_object_id("ZTF23aazjwtl")

print (f'RA {locus.coordinates.ra.to_string(unit=u.hour, sep=":", precision=2)}, DEC {locus.coordinates.dec.to_string(unit=u.deg, sep=":", precision=2)}')
print (f'The locus has matches/counterparts in these legacy catalogs: {locus.catalogs}')

RA 17:54:19.34, DEC 64:13:32.22
The locus has matches/counterparts in these legacy catalogs: ['2mass_xsc', 'tns_public_objects']


## Below are examples of more advanced searches using the ElasticSearch API of ANTARES

In [11]:
from antares_client.search import search

In [19]:
## We will retrieve loci, which: 
## (1) which have been tagged "nuclear_transient"
## (2) first alert was sent by ZTF within the last month 
## (3) number of alerts (think of it as data points on its light curve) is greater than 10
## (4) latest alert is brighter than 18.5 mag

query1 = {
    "query": {
        "bool": {
            "filter": {
                "bool": {
                    "must": [
                        {
                            "range": {
                                "properties.oldest_alert_observation_time": {
                                    "gte": 60161.8123530671
                                }
                            }
                        },
                        {
                            "range": {
                                "properties.num_mag_values": {
                                    "gte": 10
                                }
                            }
                        },
                        {
                            "terms": {
                                "tags": [
                                    "nuclear_transient"
                                ]
                            }
                        },
                        {
                            "range": {
                                "properties.brightest_alert_magnitude": {
                                    "lte": 18.5
                                }
                            }
                        }
                    ]
                }
            }
        }
    }
}


In [29]:
for i,locus in enumerate(search(query1)):
    print (i+1, f"ZTF_ID {locus.properties['ztf_object_id']}, RA {locus.coordinates.ra.to_string(unit=u.hour, sep=':', precision=2)},\
    DEC {locus.coordinates.dec.to_string(unit=u.deg, sep=':', precision=2)}, its Antares link is https://antares.noirlab.edu/loci/{locus.locus_id}")

1 ZTF_ID ZTF23abaspqd, RA 9:00:26.33,    DEC 39:55:59.60, its Antares link is https://antares.noirlab.edu/loci/ANT20237rkde6tg6tqa
2 ZTF_ID ZTF23aawwcuu, RA 21:11:20.26,    DEC 31:05:42.69, its Antares link is https://antares.noirlab.edu/loci/ANT20238hk246ek921t
3 ZTF_ID ZTF23aaynddf, RA 0:48:53.72,    DEC 18:37:44.72, its Antares link is https://antares.noirlab.edu/loci/ANT2023s2pxc5r7o0j2
4 ZTF_ID ZTF23aawqydv, RA 1:17:52.44,    DEC 0:46:50.88, its Antares link is https://antares.noirlab.edu/loci/ANT2023qo98v9g8wp66
5 ZTF_ID ZTF23aaxuvkn, RA 7:17:03.17,    DEC 24:57:31.64, its Antares link is https://antares.noirlab.edu/loci/ANT20234vk5r46b40wf
6 ZTF_ID ZTF23aaxfnwp, RA 17:43:44.86,    DEC 33:31:48.68, its Antares link is https://antares.noirlab.edu/loci/ANT2023n0h5y0mmfiza
7 ZTF_ID ZTF23aawkjdx, RA 23:44:54.94,    DEC 12:21:21.45, its Antares link is https://antares.noirlab.edu/loci/ANT2023x6thwrc65f0q
8 ZTF_ID ZTF23aawnewu, RA 6:51:15.27,    DEC 43:23:19.52, its Antares link is htt

In [37]:
## In this query, we will find loci tagged "nova_test", whose latest alerts are between 17.95 and 18 mag


query2 = {
    "query": {
        "bool": {
            "filter": {
                "bool": {
                    "must": [
                        {
                            "terms": {
                                "tags": [
                                    "nova_test"
                                ]
                            }
                        },
                        {
                            "range": {
                                "properties.brightest_alert_magnitude": {
                                    "lte": 18.0,
                                    "gte": 17.95
                                }
                            }
                        }
                    ]
                }
            }
        }
    }
}


In [38]:
for i,locus in enumerate(search(query2)):
    print (i+1, f"ZTF_ID {locus.properties['ztf_object_id']}, RA {locus.coordinates.ra.to_string(unit=u.hour, sep=':', precision=2)},\
    DEC {locus.coordinates.dec.to_string(unit=u.deg, sep=':', precision=2)}, latest alert is {locus.properties['brightest_alert_magnitude']:.2f} mag,\
    its Antares link is https://antares.noirlab.edu/loci/{locus.locus_id}")

1 ZTF_ID ZTF23aavugvk, RA 0:23:56.40,    DEC 28:50:57.56, latest alert is 17.96 mag,    its Antares link is https://antares.noirlab.edu/loci/ANT2023hzwf3yeqqee0
2 ZTF_ID ZTF23aawieyi, RA 16:21:52.56,    DEC -0:57:03.56, latest alert is 17.98 mag,    its Antares link is https://antares.noirlab.edu/loci/ANT2023ddz9xzl3bur4
3 ZTF_ID ZTF18abgivbz, RA 18:06:16.94,    DEC 25:21:37.84, latest alert is 17.98 mag,    its Antares link is https://antares.noirlab.edu/loci/ANT2018enfoq
4 ZTF_ID ZTF23aaqkpwb, RA 22:11:08.57,    DEC -24:27:56.49, latest alert is 17.97 mag,    its Antares link is https://antares.noirlab.edu/loci/ANT20236t9eur7u6xht
5 ZTF_ID ZTF19adcppsg, RA 18:34:43.69,    DEC -25:18:21.42, latest alert is 17.95 mag,    its Antares link is https://antares.noirlab.edu/loci/ANT2020gzzqy
6 ZTF_ID ZTF20abpiaxy, RA 0:44:55.16,    DEC 36:41:32.41, latest alert is 17.99 mag,    its Antares link is https://antares.noirlab.edu/loci/ANT2020nhj6g
7 ZTF_ID ZTF23aaxvppp, RA 18:11:32.91,    DEC -18